## 1. 모듈 불러오기

In [1]:
import numpy as np
import pandas as pd
import re
import collections
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

## 2. 데이터 불러오기

In [2]:
data = pd.read_csv('/home/ryu/2023text/final/AInews_crawling_2207to2304.csv', parse_dates=['date'],usecols=[2,3,4,6])
#data.drop(['Unnamed: 0.1', 'Unnamed: 0', 'name', 'url'], axis=1, inplace=True)
data.head()

,title,date,content,url
0,"[[단독] 서울대 연구팀, 또 표절 의혹…이종호 장관 아들 1저자 논문]",2022-07-01 10:47:01,"[\n거의 같은 문장 반복해서 등장서울대, 연구진실성조사위원회 열어\n\n\n\n사...",https://n.news.naver.com/mnews/article/015/000...
1,[중부지방에 5∼40㎜ 소나기…낮 최고 35도 폭염],2022-07-01 05:39:10,[\n\n\n\n\n우산행렬[연합뉴스 자료사진] (서울=연합뉴스) 윤우성 기자...,https://n.news.naver.com/mnews/article/001/001...
2,"[삼성, 세계 첫 '3나노' 양산‥TSMC 잡을까]",2022-07-01 06:20:53,[\n\t\t\t[뉴스투데이]◀ 앵커 ▶삼성전자가 세계 최초로 3나노 기술을 적용한...,https://n.news.naver.com/mnews/article/214/000...
3,"[신한은행, ESG본부 신설…하반기 정기 인사]",2022-07-01 15:25:08,[\nAI 인사 업그레이드…금융사고 예방 위해 장기근속 직원 순환근무(서울=연합뉴스...,https://n.news.naver.com/mnews/article/001/001...
4,"[영남이공대, 미래 신산업 수요특화형 'AI·IoT' 교육]",2022-07-01 11:14:20,[\n\n\n\n\n[대구=뉴시스]김정화 기자 = 영남이공대학교는 지난달 27일부터...,https://n.news.naver.com/mnews/article/003/001...


In [3]:
data.isnull().sum()

title      0
date       0
content    0
url        0
dtype: int64

### 2.1 None값 있는 행 제거

In [4]:
data[data['title']=='[]']

,title,date,content,url
57,[],None,[],https://sports.news.naver.com/news.nhn?oid=311...
96,[],None,[],https://sports.news.naver.com/news.nhn?oid=439...
230,[],None,[],https://sports.news.naver.com/news.nhn?oid=382...
1091,[],None,[],https://sports.news.naver.com/news.nhn?oid=009...
1492,[],None,[],https://sports.news.naver.com/news.nhn?oid=442...
...,...,...,...,...
38898,[],None,[],https://sports.news.naver.com/news.nhn?oid=139...
39154,[],None,[],https://n.news.naver.com/mnews/article/629/000...
39332,[],None,[],https://sports.news.naver.com/news.nhn?oid=468...
39362,[],None,[],https://sports.news.naver.com/news.nhn?oid=347...


In [5]:
# None 없는 행 제외하고 데이터 다시 구성
data = data[data['date']!='None']

In [6]:
len(data[data['content'].isnull()])

0

### 2.2 날짜 형식 변환 및 날짜 관련 칼럼 생성

In [7]:
# 날짜를 datetime 형식으로 변환
data['date'] = pd.to_datetime(data['date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40009 entries, 0 to 40192
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    40009 non-null  object        
 1   date     40009 non-null  datetime64[ns]
 2   content  40009 non-null  object        
 3   url      40009 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.5+ MB


In [8]:
# 새로운 칼럼 생성 및 값 할당
data['timeline'] = data['date'].apply(lambda x: 'before' if x < datetime.datetime(2022,11,30) else 'after')

In [9]:
# 기준 시점 전후 데이터 양 차이 비교
print('ChatGPT 이전 뉴스데이터 개수 :',len(data[data['timeline']=='before']))
print('ChatGPT 이후 뉴스데이터 개수 :',len(data[data['timeline']=='after']))

ChatGPT 이전 뉴스데이터 개수 : 18913
ChatGPT 이후 뉴스데이터 개수 : 21096


In [10]:
# 연도/월 칼럼 생성
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [11]:
data.head()

,title,date,content,url,timeline,year,month
0,"[[단독] 서울대 연구팀, 또 표절 의혹…이종호 장관 아들 1저자 논문]",2022-07-01 10:47:01,"[\n거의 같은 문장 반복해서 등장서울대, 연구진실성조사위원회 열어\n\n\n\n사...",https://n.news.naver.com/mnews/article/015/000...,before,2022,7
1,[중부지방에 5∼40㎜ 소나기…낮 최고 35도 폭염],2022-07-01 05:39:10,[\n\n\n\n\n우산행렬[연합뉴스 자료사진] (서울=연합뉴스) 윤우성 기자...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
2,"[삼성, 세계 첫 '3나노' 양산‥TSMC 잡을까]",2022-07-01 06:20:53,[\n\t\t\t[뉴스투데이]◀ 앵커 ▶삼성전자가 세계 최초로 3나노 기술을 적용한...,https://n.news.naver.com/mnews/article/214/000...,before,2022,7
3,"[신한은행, ESG본부 신설…하반기 정기 인사]",2022-07-01 15:25:08,[\nAI 인사 업그레이드…금융사고 예방 위해 장기근속 직원 순환근무(서울=연합뉴스...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
4,"[영남이공대, 미래 신산업 수요특화형 'AI·IoT' 교육]",2022-07-01 11:14:20,[\n\n\n\n\n[대구=뉴시스]김정화 기자 = 영남이공대학교는 지난달 27일부터...,https://n.news.naver.com/mnews/article/003/001...,before,2022,7


## 3. 전처리 함수

### 3.1 대괄호, 줄바꿈, 탭 등 제거

In [12]:
# 대괄호 제거
def remove_bracket(text):
    text = text[1:-1]
    return text

In [13]:
# 줄바꿈과 탭 표시 제거
def remove_n(text):
    text = text.replace('\n', '').replace('\t', '')
    return text

In [14]:
# title과 content에 대괄호 제거
data[['title', 'content']] = data[['title', 'content']].applymap(lambda x: remove_bracket(x))
data.head()

,title,date,content,url,timeline,year,month
0,"[단독] 서울대 연구팀, 또 표절 의혹…이종호 장관 아들 1저자 논문",2022-07-01 10:47:01,"\n거의 같은 문장 반복해서 등장서울대, 연구진실성조사위원회 열어\n\n\n\n사진...",https://n.news.naver.com/mnews/article/015/000...,before,2022,7
1,중부지방에 5∼40㎜ 소나기…낮 최고 35도 폭염,2022-07-01 05:39:10,\n\n\n\n\n우산행렬[연합뉴스 자료사진] (서울=연합뉴스) 윤우성 기자 ...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
2,"삼성, 세계 첫 '3나노' 양산‥TSMC 잡을까",2022-07-01 06:20:53,\n\t\t\t[뉴스투데이]◀ 앵커 ▶삼성전자가 세계 최초로 3나노 기술을 적용한 ...,https://n.news.naver.com/mnews/article/214/000...,before,2022,7
3,"신한은행, ESG본부 신설…하반기 정기 인사",2022-07-01 15:25:08,\nAI 인사 업그레이드…금융사고 예방 위해 장기근속 직원 순환근무(서울=연합뉴스)...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
4,"영남이공대, 미래 신산업 수요특화형 'AI·IoT' 교육",2022-07-01 11:14:20,\n\n\n\n\n[대구=뉴시스]김정화 기자 = 영남이공대학교는 지난달 27일부터 ...,https://n.news.naver.com/mnews/article/003/001...,before,2022,7


In [15]:
# content에 줄바꿈 표시 제거
data['content'] = data['content'].apply(lambda x: remove_n(x))
data.head()

,title,date,content,url,timeline,year,month
0,"[단독] 서울대 연구팀, 또 표절 의혹…이종호 장관 아들 1저자 논문",2022-07-01 10:47:01,"거의 같은 문장 반복해서 등장서울대, 연구진실성조사위원회 열어사진=한경DB서울대 ...",https://n.news.naver.com/mnews/article/015/000...,before,2022,7
1,중부지방에 5∼40㎜ 소나기…낮 최고 35도 폭염,2022-07-01 05:39:10,우산행렬[연합뉴스 자료사진] (서울=연합뉴스) 윤우성 기자 = 금요일인 1일은...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
2,"삼성, 세계 첫 '3나노' 양산‥TSMC 잡을까",2022-07-01 06:20:53,[뉴스투데이]◀ 앵커 ▶삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하...,https://n.news.naver.com/mnews/article/214/000...,before,2022,7
3,"신한은행, ESG본부 신설…하반기 정기 인사",2022-07-01 15:25:08,AI 인사 업그레이드…금융사고 예방 위해 장기근속 직원 순환근무(서울=연합뉴스) 신...,https://n.news.naver.com/mnews/article/001/001...,before,2022,7
4,"영남이공대, 미래 신산업 수요특화형 'AI·IoT' 교육",2022-07-01 11:14:20,[대구=뉴시스]김정화 기자 = 영남이공대학교는 지난달 27일부터 7월1일까지 제주도...,https://n.news.naver.com/mnews/article/003/001...,before,2022,7


### 3.2 기사 제목 전처리

In [16]:
# 제목 전처리 
def title_processing(text):
    new_text = re.sub(r'\[.*\]', '', text) # 괄호 안 내용 제거
    new_text = re.sub(r'\(.*\)', '', new_text)
    new_text = re.sub(r'[^ a-zA-Zㄱ-ㅎ가-힣0-9.]', ' ', new_text) # 띄어쓰기와 한글 제외 모두 제거
    new_text = re.sub(r"^\s+", '', new_text) # space from start 제거
    new_text = re.sub(r'\s+$', '', new_text) # space from the end 제거
    new_text = re.sub(r'\s{2,}', ' ', new_text) # 다중 공백 -> 하나의 공백으로

    return new_text

In [17]:
# test = '[단독] 어쩌구 23028 so did it work 저쩌구'
# title_processing(test)

In [18]:
data['title'] = data['title'].apply(lambda x: title_processing(x))

### 3.3 기사 내용 전처리

In [47]:
# pd.set_option('display.max_rows', 20)

In [19]:
# 내용 전처리

def clean_content(text):
    new_text = re.sub(r'\[.*?\]', '', text) # 대괄호와 괄호 사이 내용 제거
    new_text = re.sub(r'\(.*?\)', '', new_text) # 소괄호와 괄호 사이 내용 제거
    new_text = re.sub(r'◀.*?▶', '', new_text)

    new_text = re.sub(r'^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', '', new_text) # 이메일 주소 제거
    new_text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", ' ', new_text) # http로 시작되는 url
    new_text = re.sub(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", ' ', new_text) # http로 시작되지 않는 url
    new_text = re.sub(r'\d{2,3}-\d{3,4}-\d{3,4}', '', new_text) # 전화번호 제거
    new_text = re.sub(r'\w{2,3} 기자', '', new_text) # 기자 이름 제거 (과정 중에 불필요하게 날아가는 단어들이 있을 수 있음)

    new_text = re.sub(r'[^ a-zA-Zㄱ-ㅎ가-힣0-9.]', ' ', new_text) # 띄어쓰기, 한글, 영문 제외 모두 제거 (특수문자 제거)

    new_text = re.sub(r"^\s+", '', new_text) # space from start 제거
    new_text = re.sub(r'\s+$', '', new_text) # space from the end 제거
    new_text = re.sub(r'\s{2,}', ' ', new_text) # 다중 공백 -> 하나의 공백으로
    
    
    return new_text

In [20]:
data['content'] = data['content'].apply(lambda x: clean_content(x))

In [21]:
# 추가 처리
data['content'] = data['content'].str.replace('사진제공', '')
data['content'] = data['content'].str.replace('사진 제공', '')
data['content'] = data['content'].str.replace('세계일보 자료사진', '')
data['content'] = data['content'].str.replace('사진 연합뉴스', '')
data['content'] = data['content'].str.replace('이미지', '')
data['content'] = data['content'].str.replace('재판매 및 DB 금지', '')
data['content'] = data['content'].str.replace('재배포 및 DB 금지', '')
data['content'] = data['content'].str.replace('무단 전재 재배포 금지', '')
data['content'] = data['content'].str.replace('Copyright 뉴스1 All rights reserved', '')
data['content'] = data['content'].str.replace('Copyright', '')
data['content'] = data['content'].str.replace('All rights reserved', '')
data['content'] = data['content'].str.replace('News1', '')
data['content'] = data['content'].str.replace('뉴스1', '')
data['content'] = data['content'].apply(lambda x: re.sub(r'KBS 뉴스 \w{2,4}입니다\.', '', str(x)))
data['content'] = data['content'].apply(lambda x: re.sub(r'촬영기자 \w{2,4}', '', str(x)))

In [22]:
len(data[data['content'].isnull()])

0

In [23]:
data.dropna(subset=['content'], axis=0, inplace=True)

In [24]:
data['content']

0        거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
1        우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를 ...
2        삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다.경쟁사인...
3        AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일 자...
4        영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서 ...
                               ...                        
40188    가정의 달 봄 행사와 다양한 프로그램 마련리얼로봇 등 보강해 체감형 로봇랜드로 탈바...
40189    임승혁 KT 헬스케어사업단 단장이 감염병 대비를 위한 차세대 방역 연구 성과공유회에...
40190    AI와 통신데이터를 활용해 차세대 방역에 대비하는 감염병 경로예측 모델이 개발됐다....
40191    창업자 등 면담하고 시연도 참관이창양 산업통상자원부 장관이 28일 미국 매사추세츠주...
40192    어린이 날 연휴 로봇마차 퍼레이드 솜 사탕 로봇 라면 끓이는 로봇 도 선보여로봇 소...
Name: content, Length: 40009, dtype: object

In [25]:
data.to_csv('/home/ryu/2023text/final/for_textrank.csv', index=False, encoding='utf-8')

## 4. 문장 단위로 split

In [75]:
# 필요한 칼럼만 가져오기
split_df = data[['title', 'content', 'timeline', 'year', 'month']]
split_df

,title,content,timeline,year,month
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,before,2022,7
1,중부지방에 5 40 소나기 낮 최고 35도 폭염,우산행렬 금요일인 1일은 늦은 오후부터 저녁 사이 중부지방을 중심으로 천둥 번개를 ...,before,2022,7
2,삼성 세계 첫 3나노 양산 TSMC 잡을까,삼성전자가 세계 최초로 3나노 기술을 적용한 반도체를 생산하기 시작했습니다.경쟁사인...,before,2022,7
3,신한은행 ESG본부 신설 하반기 정기 인사,AI 인사 업그레이드 금융사고 예방 위해 장기근속 직원 순환근무 신한은행은 1일 자...,before,2022,7
4,영남이공대 미래 신산업 수요특화형 AI IoT 교육,영남이공대학교는 지난달 27일부터 7월1일까지 제주도 호텔골든데이지 서귀포오션에서 ...,before,2022,7
...,...,...,...,...,...
40188,마산 로봇랜드 테마파크 볼거리 풍성 운영 활성화,가정의 달 봄 행사와 다양한 프로그램 마련리얼로봇 등 보강해 체감형 로봇랜드로 탈바...,after,2023,4
40189,KT 기지국 활용 역학조사 효과 입증 감염병 대응 연구 성과 공유,임승혁 KT 헬스케어사업단 단장이 감염병 대비를 위한 차세대 방역 연구 성과공유회에...,after,2023,4
40190,통신데이터로 제2코로나 막는다 KT 예측모델 개발 완료,AI와 통신데이터를 활용해 차세대 방역에 대비하는 감염병 경로예측 모델이 개발됐다....,after,2023,4
40191,보스톤 다이내믹스 찾은 이창양 로봇 기술협력 우수 사례,창업자 등 면담하고 시연도 참관이창양 산업통상자원부 장관이 28일 미국 매사추세츠주...,after,2023,4


In [78]:
# .를 기준으로 쪼개기
split_df['content_split'] = split_df['content'].str.split('.')

/tmp/ipykernel_10041/2095326785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['content_split'] = split_df['content'].str.split('.')


In [79]:
# 쪼개진 결과 보기
split_df['content_split'][0]

['거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공지능 연구팀의 표절 사실이 드러난 가운데 같은 연구팀 논문에 또다른 표절 의혹이 제기됐다',
 ' 이번 논문은 이종호 과학기술정보통신부 장관 아들인 이모씨가 1저자로 낸 논문이다',
 ' 44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로 표절이 제기된 논문은 지난해 6월 발표된 Energy efficient Knowledge Distillation for Spiking Neural Networks 다',
 ' 앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공학부 교수 연구실에서 나온 논문이다',
 ' 이 장관의 아들인 서울대 박사과정생 이모씨가 1저자고 윤 교수가 교신저자로 이름을 올렸다',
 ' 나머지 공저자 3명 중 2명은 먼저 표절 의혹이 제기됐던 논문에도 참여했던 대학원생들이다',
 ' 이 또 다른 논문에 표절 의혹이 제기된 건 지난달 27일 서울대 온라인 커뮤니티에서다',
 ' 표절 대상이라고 지목된 논문은 두 개다',
 ' 하나는 홍콩계 AI 기업인 센스타임 소속 연구자 등이 2019년 발표한 Knowledge Distillation via Route Constrained Optimization 이라는 논문이다',
 ' 이씨의 논문 3쪽 2',
 '2절 첫 번째 문장은 센스타임 논문에서 26개 단어로 이뤄진 한 문장과 거의 같다',
 ' 원문의 is 를 has been 으로 바꾼 것 말고는 완벽히 똑같은 문장이다',
 ' 센스타임 논문에 대한 인용 표시도 없다',
 '이 부분은 관련 선행 연구를 설명하는 대목으로 코넬대 연구진이 2006년 발표한 논문을 인용 표시했다',
 ' 이씨의 논문은 센스타임 연구진이 코넬대 연구진의 논문을 자신들의 언어로 새로이 요약 인용한 대목을 그대로 베껴온 것으로 보인다',
 '이씨의 논문과 센스타임의 논문이 동일하게 코넬대 연구진의 논문을 인용했다고 하더라도 문장이 똑같

In [ ]:
# 각 행의 list를 칼럼으로 쪼개기
cont_sp = split_df['content_split'].apply(lambda x: pd.Series(x))
cont_sp.head(3)

In [28]:
# 칼럼을 밑으로 나열하기
cont_sp.stack()

0      0     거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
       1          이번 논문은 이종호 과학기술정보통신부 장관 아들인 이모씨가 1저자로 낸 논문이다
       2      44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로...
       3      앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공...
       4      이 장관의 아들인 서울대 박사과정생 이모씨가 1저자고 윤 교수가 교신저자로 이름을...
                                   ...                        
40192  9                         로봇을 소재로 한 다양한 교육 체험프로그램도 마련한다
       10    로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 ...
       11    로봇체험관을 활용한 해설이 있는 에듀투어 에서는 유 초 중 고 학생을 대상으로 로봇...
       12    류명현 경남도 산업통상국장은 매년 리얼로봇을 추가 도입하고 다양한 로봇 콘텐츠를 보...
       13                                                     
Length: 870601, dtype: object

In [29]:
# 인덱스 제거
cont_sp.stack().reset_index(level=1, drop=True)

0        거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
0             이번 논문은 이종호 과학기술정보통신부 장관 아들인 이모씨가 1저자로 낸 논문이다
0         44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로...
0         앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공...
0         이 장관의 아들인 서울대 박사과정생 이모씨가 1저자고 윤 교수가 교신저자로 이름을...
                               ...                        
40192                        로봇을 소재로 한 다양한 교육 체험프로그램도 마련한다
40192    로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 ...
40192    로봇체험관을 활용한 해설이 있는 에듀투어 에서는 유 초 중 고 학생을 대상으로 로봇...
40192    류명현 경남도 산업통상국장은 매년 리얼로봇을 추가 도입하고 다양한 로봇 콘텐츠를 보...
40192                                                     
Length: 870601, dtype: object

In [30]:
# 데이터프레임 형태로 변환
cont_sp = cont_sp.stack().reset_index(level=1, drop=True).to_frame('sentence')
cont_sp

,sentence
0,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
0,이번 논문은 이종호 과학기술정보통신부 장관 아들인 이모씨가 1저자로 낸 논문이다
0,44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로...
0,앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공...
0,이 장관의 아들인 서울대 박사과정생 이모씨가 1저자고 윤 교수가 교신저자로 이름을...
...,...
40192,로봇을 소재로 한 다양한 교육 체험프로그램도 마련한다
40192,로봇특화 교육 및 진로체험으로 구성된 로봇체험교실 에서는 로봇랜드 대표 놀이시설인 ...
40192,로봇체험관을 활용한 해설이 있는 에듀투어 에서는 유 초 중 고 학생을 대상으로 로봇...
40192,류명현 경남도 산업통상국장은 매년 리얼로봇을 추가 도입하고 다양한 로봇 콘텐츠를 보...


In [31]:
# 기존의 데이터와 병합
data = data.merge(cont_sp, left_index=True, right_index=True, how='left')

In [32]:
data.head()

,title,date,content,url,timeline,year,month,sentence
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,이번 논문은 이종호 과학기술정보통신부 장관 아들인 이모씨가 1저자로 낸 논문이다
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로...
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공...
0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,이 장관의 아들인 서울대 박사과정생 이모씨가 1저자고 윤 교수가 교신저자로 이름을...


In [47]:
# 인덱스를 기사 번호 칼럼으로 변경
data = data.reset_index().rename(columns={"index": "news_idx"})

In [48]:
data.head()

,news_idx,title,date,content,url,timeline,year,month,sentence
0,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...
1,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,이번 논문은 이종호 과학기술정보통신부 장관 아들인 이모씨가 1저자로 낸 논문이다
2,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,44단어 두 문장 다른 논문에서 짜깁기1일 한국경제신문 취재를 종합하면 추가적으로...
3,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,앞서 지난달 24일 표절 의혹이 제기된 논문과 마찬가지로 윤성로 서울대 전기정보공...
4,0,서울대 연구팀 또 표절 의혹 이종호 장관 아들 1저자 논문,2022-07-01 10:47:01,거의 같은 문장 반복해서 등장서울대 연구진실성조사위원회 열어사진 한경DB서울대 인공...,https://n.news.naver.com/mnews/article/015/000...,before,2022,7,이 장관의 아들인 서울대 박사과정생 이모씨가 1저자고 윤 교수가 교신저자로 이름을...


In [39]:
len(data)

870601

In [38]:
# 내용이 없는 경우 제거
len(data[data['sentence']==''])

37140

In [40]:
# None 없는 행 제외하고 데이터 다시 구성
data = data[data['sentence']!='']

In [41]:
len(data)

833461

In [42]:
# split 후 문자열 앞에 공백 있는 경우 다시 처리
def space_clean(sentence):

    n_sentence = re.sub(r"^\s+", '', sentence) # space from start 제거
    n_sentence = re.sub(r'\s+$', '', n_sentence) # space from the end 제거
    n_sentence = re.sub(r'\s{2,}', ' ', sentence) # 다중 공백 -> 하나의 공백으로
    
    return n_sentence

In [44]:
data['sentence'] = data['sentence'].apply(lambda x: space_clean(x))

### 4.1 중간데이터 저장

In [50]:
# (필요 칼럼만) 중간데이터 저장
sent_o = data[['news_idx', 'timeline', 'year', 'month', 'sentence']]
sent_o.to_csv('/home/ryu/2023text/final/AInews_sent_only_ver1.csv', index=False, encoding='utf-8')

## 참고

In [33]:
test_df = pd.DataFrame({'foo': ['a,b,c,d,e', 'd,e,f', 'h,i']})
test_df


,foo
0,"a,b,c,d,e"
1,"d,e,f"
2,"h,i"


In [35]:
split = test_df.foo.str.split(',')
split

0    [a, b, c, d, e]
1          [d, e, f]
2             [h, i]
Name: foo, dtype: object

In [36]:
split = split.apply(lambda x: pd.Series(x))
split

,0,1,2,3,4
0,a,b,c,d,e
1,d,e,f,NaN,NaN
2,h,i,NaN,NaN,NaN


In [37]:
split.stack()

0  0    a
   1    b
   2    c
   3    d
   4    e
1  0    d
   1    e
   2    f
2  0    h
   1    i
dtype: object

In [38]:
split.stack().reset_index(level=1, drop=True)

0    a
0    b
0    c
0    d
0    e
1    d
1    e
1    f
2    h
2    i
dtype: object

In [39]:
split = split.stack().reset_index(level=1, drop=True).to_frame('foo_single')
split

,foo_single
0,a
0,b
0,c
0,d
0,e
1,d
1,e
1,f
2,h
2,i


In [41]:
test_df.merge(split, left_index=True, right_index=True, how='left')

,foo,foo_single
0,"a,b,c,d,e",a
0,"a,b,c,d,e",b
0,"a,b,c,d,e",c
0,"a,b,c,d,e",d
0,"a,b,c,d,e",e
1,"d,e,f",d
1,"d,e,f",e
1,"d,e,f",f
2,"h,i",h
2,"h,i",i


In [8]:
# # 중간데이터 저장
# data.to_csv('/home/ryu/2023text/final/AInews_processed_ver2.csv', index=False, encoding='utf-8')